In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
q_a_data = pd.read_csv('/content/drive/MyDrive/Books/Ai Writting/Conversation/final_Chat_Data_GBC.csv')
q_a_data = q_a_data.drop("sentiment",axis=1)
persona_data = pd.read_csv('/content/drive/MyDrive/Books/Ai Writting/Conversation/personality.csv')
persona_data = persona_data.drop("Unnamed: 0",axis = 1)

In [4]:
q_a_data.head()

,question,answer
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.


In [5]:

persona_data.head()

,Persona,chat
0,i like to remodel homes. i like to go hunting...,"hi , how are you doing ? i am getting ready to..."
1,my mom is my best friend. i have four sisters...,"hi , how are you doing today ?\ni am spending ..."
2,i had a gig at local theater last night. i wo...,"we all live in a yellow submarine , a yellow s..."
3,i am very athletic. i wear contacts. i have b...,hi ! i work as a gourmet cook .\ni do not like...
4,i am primarily a meat eater. i am a guitar pl...,how are you doing today\nwhat do you do for ca...


In [6]:
data1 = q_a_data
data2 = persona_data

In [7]:
data2.isnull().sum()

Persona    0
chat       0
dtype: int64

In [8]:
import pandas as pd

# Select relevant columns from the datasets
data1_selected = data1[["question", "answer"]]
data2_selected = data2[["Persona", "chat"]]

# Join question and persona, answer and chat into single columns
data1_selected["text"] = data1_selected["question"]
data1_selected.loc[len(data1_selected)-1, "text"] += " " + data2_selected["Persona"].values[0]
data1_selected["response"] = data1_selected["answer"]
data1_selected.loc[len(data1_selected)-1, "response"] += " " + data2_selected["chat"].values[0]

# Drop unnecessary columns
data1_selected = data1_selected[["text", "response"]]

# Concatenate the datasets
combined_data = pd.concat([data1_selected, data2_selected.iloc[1:]])

# Reset the index of the combined dataset
combined_data.reset_index(drop=True, inplace=True)

# Replace NaN values with empty string
combined_data.fillna("", inplace=True)

combined_data['User'] = combined_data['text'] + combined_data['Persona']
combined_data['Reply'] = combined_data['response'] + combined_data['chat']

In [9]:
cols_drop = ["text" ,"response", "Persona","chat"]
combined_data = combined_data.drop(cols_drop,axis=1)
combined_data.tail()

,User,Reply
12658,my favorite food is fried chicken. i have a t...,"good evening , i have been having a terrible t..."
12659,i love fast food. i am a stay at home mom. i ...,hi there how is work\ni do not work these days...
12660,my family and i go camping every month. i am ...,"hello , i have a daughter who is in kindergart..."
12661,i have red hair. i work at a retail store. i ...,hi . how are you doing ?\ni am doing well . ju...
12662,i m applying for publishing jobs. the only au...,hello ! how are you today ?\nhello i am good ....


In [10]:
combined_data.to_csv('Final_OUT_Conv_Data.csv')

#Pre Processing Dataset

**Text Cleaning:** Remove any unnecessary characters, symbols, or special characters that do not contribute to the meaning of the text. This includes removing HTML tags, URLs, or any noise specific to your dataset.

**Lowercasing:** Convert the text to lowercase to ensure consistency and avoid treating words with different casing as different entities.

**Tokenization:** Split the text into individual tokens, which can be words or subword units. Tokenization helps the model understand the structure of the text and build its vocabulary.

**Special Tokens:** Add special tokens to denote the beginning and end of sentences or prompts, as well as special tokens for padding or masking if necessary.

**Sequence Length Limit:** Set a maximum sequence length for the input data. If a sequence exceeds this limit, it needs to be truncated or split into smaller segments to fit within the model's memory constraints.

**Handling Long Texts:** For long texts that cannot fit into a single sequence, you can use techniques like sliding window or recursive encoding to process the text in smaller chunks.

**Data Encoding:** Map the tokens to their corresponding integer IDs according to the model's vocabulary. This step transforms the text data into a numerical representation that can be processed by the model.

**Data Formatting:** Organize the encoded data into input-output pairs suitable for the model. For example, for autoregressive models like GPT, the input sequence can be followed by the target sequence shifted by one position.

**Masking:** Apply attention masks or padding masks to indicate which tokens are valid input and which should be ignored during training. This helps the model focus on relevant tokens and handle varying sequence lengths.

**Data Batching:** Group the input-output pairs into batches for more efficient training. Batching allows parallel processing and improves training speed.

In [11]:
!pip install transformers tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 127.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.9 MB/s eta 0:00:00


In [12]:
import re
import torch
from transformers import GPT2Tokenizer

In [16]:
data = combined_data

##Cleaning Text


1. Removing leading and trailing spaces using the strip method.

2. Removing HTML tags using a regular expression pattern (<.*?>) and the re.sub function.

3. Removing URLs using a regular expression pattern (http\S+|www\S+|https\S+) and the re.sub function.

4. Removing special characters or symbols using a regular expression pattern ([^a-zA-Z0-9\s]) and the re.sub function.

5. Normalizing whitespace using a regular expression pattern (\s+) and the re.sub function.

6. Replacing multiple consecutive punctuation marks (e.g., "!!!" or "??") with a single one using a regular expression pattern (([!?.]){2,}) and the re.sub function.

7. Replacing numbers with a special token (e.g., "NUM") using a regular expression pattern (\d+) and the re.sub function.

8. Lowering the text to small letters






In [14]:
import re

def clean_text(text):
    # Remove leading/trailing spaces
    text = text.strip()
    # Remove HTML tags
    text = re.sub(r"<.*?>", "", text)
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    # Remove special characters or symbols
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)
    # Replace multiple consecutive punctuation marks with a single one
    # text = re.sub(r"([!?.]){2,}", r"\1", text)
    text = re.sub(r"([!?.]){2,}", r"", text)
    # Replace numbers with a special token
    text = re.sub(r"\d+", "NUM", text)

    return text

# Apply text cleaning to "User" and "Reply" columns
data["User"] = data["User"].apply(clean_text)
data["Reply"] = data["Reply"].apply(clean_text)

##Tokenization

In [15]:
# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
data["User_tokens"] = data["User"].apply(tokenizer.tokenize)
data["Reply_tokens"] = data["Reply"].apply(tokenizer.tokenize)

In [16]:
# Step 4: Special Tokens
special_tokens = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
    "mask_token": "<mask>"
}
tokenizer.add_special_tokens(special_tokens)

4

In [17]:
# Step 6: Data Encoding
data["User_ids"] = data["User_tokens"].apply(tokenizer.convert_tokens_to_ids)
data["Reply_ids"] = data["Reply_tokens"].apply(tokenizer.convert_tokens_to_ids)

In [18]:
# Step 7: Data Formatting
formatted_data = [(user_ids, reply_ids) for user_ids, reply_ids in zip(data["User_ids"], data["Reply_ids"])]

In [19]:
# Step 8: Masking
attention_mask = [[1] * len(user_ids) for user_ids in data["User_ids"]]

In [20]:
# Step 9: Data Batching
batch_size = 32  # Set the desired batch size

In [21]:
# Pad the sequences to a fixed length
padded_input_ids = torch.nn.utils.rnn.pad_sequence([torch.tensor(ids) for ids, _ in formatted_data], batch_first=True)
padded_attention_mask = torch.nn.utils.rnn.pad_sequence([torch.tensor(mask) for _, mask in zip(data["User_ids"], attention_mask)], batch_first=True)

In [22]:
# Convert the padded sequences and attention masks into PyTorch tensors
input_ids = torch.tensor(padded_input_ids)
attention_mask = torch.tensor(padded_attention_mask)

# Create data batches
data_batches = torch.utils.data.TensorDataset(input_ids, attention_mask)
data_loader = torch.utils.data.DataLoader(data_batches, batch_size=batch_size, shuffle=True)

<ipython-input-22-c5607bf1755d>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(padded_input_ids)
<ipython-input-22-c5607bf1755d>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(padded_attention_mask)


Summary Code

In [ ]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer
from torch.nn.utils.rnn import pad_sequence



# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Step 1: Text Cleaning
# Step 2: Lowercasing
# Step 3: Tokenization
# Step 4: Special Tokens
# Step 5: Sequence Length Limit
# Step 6: Data Encoding
formatted_data = []
attention_mask = []

for user_text, reply_text in zip(data["User"], data["Reply"]):
    # Text Cleaning
    user_text = clean_text(user_text)
    reply_text = clean_text(reply_text)

    # Lowercasing
    user_text = user_text.lower()
    reply_text = reply_text.lower()

    # Tokenization
    user_tokens = tokenizer.tokenize(user_text)
    reply_tokens = tokenizer.tokenize(reply_text)

    # Special Tokens
    user_tokens = [tokenizer.bos_token] + user_tokens + [tokenizer.eos_token]
    reply_tokens = [tokenizer.bos_token] + reply_tokens + [tokenizer.eos_token]

    # Sequence Length Limit
    max_sequence_length = 128  # Set the maximum sequence length
    user_tokens = user_tokens[:max_sequence_length]
    reply_tokens = reply_tokens[:max_sequence_length]

    # Data Encoding
    user_ids = tokenizer.convert_tokens_to_ids(user_tokens)
    reply_ids = tokenizer.convert_tokens_to_ids(reply_tokens)

    formatted_data.append((user_ids, reply_ids))
    attention_mask.append([1] * len(user_ids))

# # Step 7: Data Formatting
# input_ids = pad_sequence([torch.tensor(user_ids) for user_ids, _ in formatted_data], batch_first=True)
# target_ids = pad_sequence([torch.tensor(reply_ids) for _, reply_ids in formatted_data], batch_first=True)
# attention_mask = pad_sequence([torch.tensor(mask) for mask in attention_mask], batch_first=True)

# Step 7: Data Formatting
input_ids = pad_sequence([torch.tensor(user_ids) for user_ids, _ in formatted_data], batch_first=True, padding_value=0.0)  # Set padding value to 0.0
target_ids = pad_sequence([torch.tensor(reply_ids) for _, reply_ids in formatted_data], batch_first=True, padding_value=0.0)  # Set padding value to 0.0
attention_mask = pad_sequence([torch.tensor(mask) for mask in attention_mask], batch_first=True)


# Step 8: Masking
attention_mask = attention_mask.bool()

# Step 9: Data Batching
batch_size = 32  # Set the desired batch size

# Create data batches
data_batches = torch.utils.data.TensorDataset(input_ids, attention_mask, target_ids)
data_loader = torch.utils.data.DataLoader(data_batches, batch_size=batch_size, shuffle=True)

#Model Building

In [23]:
from transformers import GPT2LMHeadModel

In [24]:
# Load the pre-trained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
# Set the device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set the model to the desired device
model = model.to(device)

In [25]:
# Set hyperparameters
learning_rate = 1e-6
num_epochs = 3

In [26]:
# Set optimizer and learning rate scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)



```
# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    total_batches = 0

    for batch_idx, (input_ids, attention_mask, target_ids) in enumerate(data_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        target_ids = target_ids.to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        total_loss += loss.item()
        total_batches += 1

    # Calculate average loss for the epoch
    average_loss = total_loss / total_batches

    # Update learning rate
    scheduler.step()

    # Print progress
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {average_loss}")

# Save the trained model
model.save_pretrained("trained_model")
tokenizer.save_pretrained("trained_model")
```



In [ ]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.nn.utils.rnn import pad_sequence

average_loss_p = []

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Step 1: Text Cleaning
# Step 2: Lowercasing
# Step 3: Tokenization
# Step 4: Special Tokens
# Step 5: Sequence Length Limit
# Step 6: Data Encoding
formatted_data = []
attention_mask = []

for user_text, reply_text in zip(data["User"], data["Reply"]):
    # Text Cleaning
    user_text = clean_text(user_text)
    reply_text = clean_text(reply_text)

    # Lowercasing
    user_text = user_text.lower()
    reply_text = reply_text.lower()

    # Tokenization
    user_tokens = tokenizer.tokenize(user_text)
    reply_tokens = tokenizer.tokenize(reply_text)

    # Special Tokens
    user_tokens = [tokenizer.bos_token] + user_tokens + [tokenizer.eos_token]
    reply_tokens = [tokenizer.bos_token] + reply_tokens + [tokenizer.eos_token]

    # Sequence Length Limit
    max_sequence_length = 60 # Set the maximum sequence length
    user_tokens = user_tokens[:max_sequence_length]
    reply_tokens = reply_tokens[:max_sequence_length]

    # Data Encoding
    user_ids = tokenizer.convert_tokens_to_ids(user_tokens)
    reply_ids = tokenizer.convert_tokens_to_ids(reply_tokens)

    formatted_data.append((user_ids, reply_ids))
    attention_mask.append([1] * len(user_ids))

# Step 7: Data Formatting
input_ids = pad_sequence([torch.tensor(user_ids) for user_ids, _ in formatted_data], batch_first=True, padding_value=0.0)  # Set padding value to 0.0
target_ids = pad_sequence([torch.tensor(reply_ids) for _, reply_ids in formatted_data], batch_first=True, padding_value=0.0)  # Set padding value to 0.0
attention_mask = pad_sequence([torch.tensor(mask) for mask in attention_mask], batch_first=True)

# Step 8: Masking
attention_mask = attention_mask.bool()

# Filter out incomplete batches
complete_batches = [idx for idx, (input_batch, target_batch) in enumerate(zip(input_ids, target_ids)) if input_batch.size(0) == target_batch.size(0)]
input_ids = input_ids[complete_batches]
target_ids = target_ids[complete_batches]
attention_mask = attention_mask[complete_batches]

# Step 9: Data Batching
batch_size = 64  # Set the desired batch size

if len(input_ids) > 0:
    # Create data batches
    data_batches = torch.utils.data.TensorDataset(input_ids, attention_mask, target_ids)
    data_loader = torch.utils.data.DataLoader(data_batches, batch_size=batch_size, shuffle=True)

    # Load the pre-trained GPT-2 model and tokenizer
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    # Set the device for training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Set the model to the desired device
    model = model.to(device)
    # Set hyperparameters
    learning_rate = 1e-4
    num_epochs = 150
    # Set optimizer and learning rate scheduler
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

   # Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    total_batches = 0

    for batch_idx, (input_ids, attention_mask, target_ids) in enumerate(data_loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        target_ids = target_ids.to(device)

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=target_ids)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        total_loss += loss.item()
        total_batches += 1

        # Print progress
        print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_idx+1}/{len(data_loader)} | Loss: {loss.item()}")

    # Calculate average loss for the epoch
    average_loss = total_loss / total_batches
    average_loss_p.append(average_loss)
    # Update learning rate
    scheduler.step()

    # Print progress
    print(f"Epoch {epoch+1}/{num_epochs} | Average Loss: {average_loss}")


    # Save the trained model
    model.save_pretrained("trained_model")
    tokenizer.save_pretrained("trained_model")
else:
    print("No complete batches remaining after filtering.")


Streaming output truncated to the last 5000 lines.
Epoch 10/150 | Batch 109/198 | Loss: 3.6907105445861816
Epoch 10/150 | Batch 110/198 | Loss: 3.5099165439605713
Epoch 10/150 | Batch 111/198 | Loss: 4.141637802124023
Epoch 10/150 | Batch 112/198 | Loss: 3.4878971576690674
Epoch 10/150 | Batch 113/198 | Loss: 3.7158408164978027
Epoch 10/150 | Batch 114/198 | Loss: 3.8053669929504395
Epoch 10/150 | Batch 115/198 | Loss: 3.6551880836486816
Epoch 10/150 | Batch 116/198 | Loss: 3.4025776386260986
Epoch 10/150 | Batch 117/198 | Loss: 3.5225138664245605
Epoch 10/150 | Batch 118/198 | Loss: 3.7576112747192383
Epoch 10/150 | Batch 119/198 | Loss: 3.705986738204956
Epoch 10/150 | Batch 120/198 | Loss: 3.6353344917297363
Epoch 10/150 | Batch 121/198 | Loss: 3.598421096801758
Epoch 10/150 | Batch 122/198 | Loss: 3.848510980606079
Epoch 10/150 | Batch 123/198 | Loss: 3.8709256649017334
Epoch 10/150 | Batch 124/198 | Loss: 4.4435505867004395
Epoch 10/150 | Batch 125/198 | Loss: 3.179626941680908
Ep

In [ ]:
path = '/content/drive/MyDrive/Books/trained_model'

# Save the trained model
model.save_pretrained(path)

# Save the tokenizer
tokenizer.save_pretrained("path")

# Save the model configuration
model.config.save_pretrained(path)

# Save other relevant components
torch.save(optimizer.state_dict(),path + "/optimizer.pth")
torch.save(scheduler.state_dict(), path+"/scheduler.pth")

In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Books/trained_model")

In [ ]:
for i in range(0, len(average_loss_p)):
    loss = str(average_loss_p[i])
    print("For Epoch {} Avg loss is {}".format(i + 1, loss))

#Testing

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('trained_model')
tokenizer = GPT2Tokenizer.from_pretrained('trained_model')

# Set the device for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the maximum length of generated text
max_length = 50

# Set the prompt or input text
prompt = "thanks"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: thanks. you always end stories


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('trained_model')
tokenizer = GPT2Tokenizer.from_pretrained('trained_model')

# Set the device for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the maximum length of generated text
max_length = 50

# Set the prompt or input text
prompt = "chat with me"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: chat with me for doing!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('trained_model')
tokenizer = GPT2Tokenizer.from_pretrained('trained_model')

# Set the device for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the maximum length of generated text
max_length = 50

# Set the prompt or input text
prompt = "do you have any sentiments to find"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: do you have any sentiments to find!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('trained_model')
tokenizer = GPT2Tokenizer.from_pretrained('trained_model')

# Set the device for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set the maximum length of generated text
max_length = 50

# Set the prompt or input text
prompt = "describe a story for me on any new topic"

# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Generate text
output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text: describe a story for me on any new topic you could too to the the the the a the the gets is no to no that the no on be no to the do to women doctor on what to to to wear to! for to draw a




```
from transformers import GPT2LMHeadModel

model_path = "trained_model"  # Path to the saved model directory

# Load the pretrained model
model = GPT2LMHeadModel.from_pretrained(model_path)

# Access the model configuration
config = model.config

# Print the values
print("Layers:", config.num_hidden_layers)
print("d_model:", config.hidden_size)
print("head_dim:", config.n_head)
print("Vocabulary:", config.vocab_size)
print("Sequence length:", config.max_position_embeddings)

```



 the code loads a pretrained GPT-2 model and fine-tunes it using the provided dataset. The model is trained to generate appropriate replies given user inputs. The code follows a typical training loop, optimizing the model's parameters using the Adam optimizer and adjusting the learning rate during training

#Comparing GPT2 Vs Trained Model

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the trained model and tokenizer
trained_model = GPT2LMHeadModel.from_pretrained("trained_model")
trained_tokenizer = GPT2Tokenizer.from_pretrained("trained_model")

# Load the original GPT-2 model and tokenizer
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set the device for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the models to the desired device
trained_model = trained_model.to(device)
gpt2_model = gpt2_model.to(device)

# Set the maximum length for generated responses
max_length = 100

# Set the input prompt
input_prompt = "Hello, how are you?"

# Generate response using the trained model
input_ids = trained_tokenizer.encode(input_prompt, return_tensors="pt").to(device)
trained_output = trained_model.generate(input_ids, max_length=max_length, num_return_sequences=1)
trained_response = trained_tokenizer.decode(trained_output[0], skip_special_tokens=True)

# Generate response using the GPT-2 model
input_ids = gpt2_tokenizer.encode(input_prompt, return_tensors="pt").to(device)
gpt2_output = gpt2_model.generate(input_ids, max_length=max_length, num_return_sequences=1)
gpt2_response = gpt2_tokenizer.decode(gpt2_output[0], skip_special_tokens=True)

# Print the generated responses
print("Trained Model Response:", trained_response)
print("GPT-2 Model Response:", gpt2_response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Trained Model Response: Hello, how are you?!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
GPT-2 Model Response: Hello, how are you?

I'm a little bit of a nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd. I'm a big nerd


#Version 2 AI
with additional data reddit conv's and qa dataset

In [11]:
reddit_conv = pd.read_csv('/content/drive/MyDrive/Books/Ai Writting/Conversation/fixed_reddit.csv')
data3 = reddit_conv

In [12]:
data3 = data3.drop('Unnamed: 0',axis = 1)
data3.head(10)

,Question,Reply
0,What kind of phone(s) do you guys have?,I have a pixel. It's pretty great. Much better...
1,Does it really charge all the way in 15 min?,What kind of phone(s) do you guys have?
2,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?
3,What kind of phone(s) do you guys have?,"LG Optimus V. I know, it's old."
4,My friend told me to kill myself :/,Don't kill yourself OP.
5,My friend told me to kill myself :/,"Fuck those losers, I'll /we'll be your friend"
6,I just won a state championship for debate.,No you didn't!
7,Huh?,No you didn't!
8,So I got a girlfriend the other day?,CONGRATS! Hope you are as happy as you could p...
9,So I got a girlfriend the other day?,That's so lovely!! Basically my gay-ass goals.




```
import pandas as pd

# Read the dataset
data = data3

# Fix the dataset columns
fixed_data = []
num_rows = len(data)


for i in range(len(data)-1):
    if i % 2 == 0:
        question = data.iloc[i, 0]
        if i+1 < len(data):
            reply = data.iloc[i+1, 0]
            fixed_data.append((question, reply))
        else:
            break

# Create a new DataFrame with the fixed data
fixed_reddit = pd.DataFrame(fixed_data, columns=["Question", "Reply"])

fixed_reddit.head()
```



In [13]:
from tqdm.notebook import tqdm
import re

def clean_text(text):
    # Remove leading/trailing spaces
    text = text.strip()
    # Remove HTML tags
    text = re.sub(r"<.*?>", "", text)
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    # Remove special characters or symbols
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)
    # Replace multiple consecutive punctuation marks with a single one
    text = re.sub(r"'[\/:;-_+@&!?$()<>.,@#%^&*]'","", text)
    # Replace numbers with a special token
    text = re.sub(r"\d+", "NUM", text)

    return text

In [17]:
import pandas as pd

# Select relevant columns from the datasets
data1_selected = data[["User", "Reply"]]
data2_selected = data3[["Question", "Reply"]]

# Join User and Question, Reply and Reply into single columns
data1_selected["text"] = data1_selected["User"]
data1_selected.loc[len(data1_selected)-1, "text"] += " " + data2_selected["Question"].values[0]
data1_selected["response"] = data1_selected["Reply"]
data1_selected.loc[len(data1_selected)-1, "response"] += " " + data2_selected["Reply"].values[0]

# Drop unnecessary columns
data1_selected = data1_selected[["text", "response"]]

# Concatenate the datasets
new_combined_data = pd.concat([data1_selected, data2_selected.iloc[1:]])

# Reset the index of the combined dataset
new_combined_data.reset_index(drop=True, inplace=True)

# Replace NaN values with empty string
new_combined_data.fillna("", inplace=True)

# Print the combined dataset
new_combined_data.head()

,text,response,Question,Reply
0,"hi, how are you doing?",i'm fine. how about yourself?,,
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.,,
2,i'm pretty good. thanks for asking.,no problem. so how have you been?,,
3,no problem. so how have you been?,i've been great. what about you?,,
4,i've been great. what about you?,i've been good. i'm in school right now.,,


In [18]:
new_combined_data['User'] = new_combined_data['text'] + new_combined_data['Question']
new_combined_data['Replies'] = new_combined_data['response'] + new_combined_data['Reply']

In [19]:
cols = ["text","response","Question" ,"Reply"]
new_combined_data = new_combined_data.drop(cols,axis=1)
new_combined_data.to_csv('check_now.csv')

In [20]:
new_combined_data.shape

(40810, 2)

In [34]:
new_combined_data.head()

,User,Replies
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.


In [24]:
import pandas as pd

# Read the CSV
data4 = pd.read_csv('/content/drive/MyDrive/Books/Ai Writting/Conversation/oasst1-train.csv')


In [25]:
colss = ["message_id","parent_id","user_id","created_date","review_count",
         "review_result","deleted","rank","synthetic","model_name",
         "detoxify","message_tree_id","tree_state","labels","emojis"]
data4 = data4.drop(colss,axis=1)
# Filter the DataFrame where lang is en
data4 = data4[data4['lang'] == 'en']

In [26]:
data4 = data4.drop("lang",axis=1)
data4.head()

,text,role
0,Can you write a short introduction about the r...,prompter
1,"""Monopsony"" refers to a market structure where...",assistant
2,Now explain it to a dog,prompter
3,Monopsony is a market structure in which there...,assistant
4,How can one fight back when a monospony had be...,prompter


In [27]:
# Separate the text based on the roles and create new columns
assistant_text = data4.loc[data4['role'] == 'assistant', 'text'].reset_index(drop=True)
prompter_text = data4.loc[data4['role'] == 'prompter', 'text'].reset_index(drop=True)

# Create a new DataFrame with assistant and prompter columns
data4_formated = pd.DataFrame({
    'assistant': assistant_text,
    'prompter': prompter_text
})
data4_formated.head()

,assistant,prompter
0,"""Monopsony"" refers to a market structure where...",Can you write a short introduction about the r...
1,Monopsony is a market structure in which there...,Now explain it to a dog
2,Monopsony refers to a market structure where t...,How can one fight back when a monospony had be...
3,Here are some potential regulatory options to ...,What can be done at a regulatory level to ensu...
4,Regulatory intervention can be used to address...,I would imagine this is similar or even the sa...


In [32]:
data4_formated = data4_formated.dropna()
data4_formated.shape

(15200, 2)

In [35]:
import pandas as pd

# Select relevant columns from the datasets
data1_selected = new_combined_data[["User", "Replies"]]
data2_selected = data4_formated[["prompter","assistant"]]

# Join text and assistant, response and prompter into single columns
data1_selected["text"] = data1_selected["User"]
data1_selected.loc[len(data1_selected)-1, "User"] += " " + data2_selected["prompter"].values[0]
data1_selected["Reply"] = data1_selected["Replies"]
data1_selected.loc[len(data1_selected)-1, "Replies"] += " " + data2_selected["assistant"].values[0]


# Concatenate the datasets
new_combined_data = pd.concat([data1_selected, data2_selected.iloc[1:]])

# Reset the index of the combined dataset
new_combined_data.reset_index(drop=True, inplace=True)

# Replace NaN values with empty string
new_combined_data.fillna("", inplace=True)

In [38]:
new_combined_data['text'] = new_combined_data['text'] + new_combined_data['prompter']
new_combined_data['Reply'] = new_combined_data['Reply'] + new_combined_data['assistant']

In [39]:
cols = ["User","Replies","prompter","assistant"]
new_combined_data = new_combined_data.drop(cols,axis=1)

In [40]:
new_combined_data.tail()

,text,Reply
56004,I don't see any factor regarding the energy ef...,"just don't open your pc, take shower and focus..."
56005,"Is there something that I, as a home owner, ca...",You're welcome! Remember to create a conducive...
56006,How can I best stay informed about market trends?,You're welcome! :)
56007,"That's not really that helpful, can you elabor...",NASA's current major goals are:\n\n1. Return h...
56008,What are the risks when you do not carry out A...,"NASA, like most federal agencies, has to opera..."


In [42]:
new_combined_data.to_csv("Final_OUT_Data_2.csv")